In [6]:
import pandas as pd
import argparse
from nilearn.glm.second_level import SecondLevelModel
from nilearn import plotting
import os
from nilearn.glm.second_level import make_second_level_design_matrix
from nilearn.maskers import NiftiMasker
from sklearn.preprocessing import StandardScaler

group = "gangnam_sad"
variable = "LSAS"
roi = "lt_precuneus"
smoothness = 6
mdmr_dir = os.path.expanduser("~/fmri_project/C-PAC/CPAC/bcb_mdmr/")
nas_dir = os.path.expanduser("/mnt/NAS2-2/data/")
MDMR_output_dir = f"{nas_dir}/SAD_gangnam_MDMR/"
fmri_prep_dir = f"{nas_dir}/SAD_gangnam_resting_2/fMRIPrep_total"
seed_anal_dir = f"{nas_dir}/SAD_gangnam_seed_based_analysis/"

regressor_df = pd.read_csv(
    f"{mdmr_dir}/input/{group}_{variable}_regressor.csv"
)

subjects_label = regressor_df["Participant"].values
# 필요한 열만 선택하여 디자인 매트릭스 생성

extra_info_subjects = pd.DataFrame({
    "subject_label": subjects_label,
    variable: regressor_df[variable],
    "sex": regressor_df["SEX"],
    "age": regressor_df["AGE"],
    "yr_edu": regressor_df["YR_EDU"],
    "mean_framewise_displacement": regressor_df["Mean_Framewise_Displacement"]
})

# Create a StandardScaler object
scaler = StandardScaler()

# List of columns to normalize
columns_to_normalize = ["age", "yr_edu", variable, "mean_framewise_displacement"]

# Normalize the specified columns
extra_info_subjects[columns_to_normalize] = scaler.fit_transform(extra_info_subjects[columns_to_normalize])

design_matrix = make_second_level_design_matrix(
    subjects_label, extra_info_subjects
)


/home/changbae/anaconda3/envs/fmrienv/lib/python3.9/site-packages/nilearn/glm/first_level/design_matrix.py:503: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.745512495985117' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  design_matrix.loc[ridx, conf_name] = confounds_value
/home/changbae/anaconda3/envs/fmrienv/lib/python3.9/site-packages/nilearn/glm/first_level/design_matrix.py:503: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-1.2635659620320283' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  design_matrix.loc[ridx, conf_name] = confounds_value
/home/changbae/anaconda3/envs/fmrienv/lib/python3.9/site-packages/nilearn/glm/first_level/design_matrix.py:503: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise 

In [7]:
z_maps = [f"{seed_anal_dir}/{smoothness}mm/corr_z-map/seed_{group}_{variable}_{roi}/sub-{subject_id}_fisher_z_img.nii.gz" for subject_id in regressor_df['Participant']]
second_level_model = SecondLevelModel(n_jobs=-1)
second_level_model = second_level_model.fit(
    z_maps,
    design_matrix=design_matrix,
)

In [8]:
p_map = second_level_model.compute_contrast(variable,output_type="p_value")
t_map = second_level_model.compute_contrast(variable, output_type="stat")



In [9]:
from nilearn import image
import numpy as np

# p-value 맵에서 0.05 이하인 부분만 마스크 생성
mask_img = image.math_img('img < 0.05', img=p_map)

# 마스크를 적용하여 t-statistic 맵 필터링
masked_t_map = image.math_img('img1 * img2', img1=t_map, img2=mask_img)

t_data = image.get_data(masked_t_map)
t_min = np.min(t_data[t_data != 0])  # 0이 아닌 값 중 최소값
t_max = np.max(t_data)

print(f"Minimum t-value (excluding zeros): {t_min}")
print(f"Maximum t-value: {t_max}")
masked_t_map.to_filename(f"{group}_{variable}_{roi}_tmap.nii.gz")

Minimum t-value (excluding zeros): 1.6863147795769546
Maximum t-value: 2.787528827809305


In [22]:
from nilearn.reporting import get_clusters_table
table, label_maps = get_clusters_table(t_map, stat_threshold=1.68, cluster_threshold=5, return_label_maps=True)
table.set_index("Cluster ID", drop=True)
table

,Cluster ID,X,Y,Z,Peak Stat,Cluster Size (mm3)
0,1,-64.375,-49.625,-19.0,2.787529,320
1,2,11.625,34.375,-19.0,2.696145,576
2,2a,7.625,26.375,-19.0,2.565766,
3,3,-0.375,26.375,-7.0,2.409346,320


In [24]:
label_maps[0].to_filename(f"{group}_{variable}_{roi}_cluster_label.nii.gz")

In [66]:
from nilearn.glm.second_level import non_parametric_inference


out_dict = non_parametric_inference(
    z_maps,
    design_matrix=design_matrix,
    second_level_contrast=variable,
    n_perm=15000,  # 500 for the sake of time. Ideally, this should be 10,000.
    two_sided_test=False,
    n_jobs=-1,
    verbose=1,
)

Fitting second level model...
Computation of second level model done in 0.13392424583435059 seconds


In [67]:
neg_log10_vfwe_pvals_img = out_dict
neg_log10_vfwe_pvals_img.to_filename(f"{group}_{variable}_{roi}_voxel_not_thresholded.nii.gz")

In [35]:
p_cut_off = 0.05
voxel_p_value_cutoff = -np.log10(p_cut_off)
mask_img = image.math_img(f'img > {voxel_p_value_cutoff}', img=neg_log10_vfwe_pvals_img)
masked_p_map = image.math_img('img1 * img2', img1=neg_log10_vfwe_pvals_img, img2=mask_img)
masked_p_map.to_filename(f"{group}_{variable}_voxel_{p_cut_off}.nii.gz")
logp_max_size = out_dict["logp_max_size"]
logp_max_t = out_dict["logp_max_t"]
cluster_cut_off = 0.05
p_cut_off = 0.005
cluster_p_value_cutoff = -np.log10(cluster_cut_off)  # p-value 0.05에 해당하는 -log10 값
voxel_p_value_cutoff = -np.log10(p_cut_off)
mask_img = image.math_img(f'img > {cluster_p_value_cutoff}', img=logp_max_size)
mask_img2 = image.math_img(f'img > {voxel_p_value_cutoff}', img=logp_max_t)
masked_p_map = image.math_img('img1 * img2', img1=mask_img, img2=mask_img2)
masked_p_map.to_filename(f"voxel_{p_cut_off}_cluster_{cluster_cut_off}_cutoff.nii.gz")